In [1]:
#Basic Spark Setup Stuff

import os
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL basic example").master("local[*]").getOrCreate()

In [2]:
# Read in our datafile of Amazon reviews of musical instruments

df = spark.read.json("reviews_Musical_Instruments_5.json.gz")

In [3]:
# Sanity check our dataset size, should be the same as in Lab #2

df.count()

10261

In [6]:
# Register our DF as a temp table
df.createOrReplaceTempView("music_reviews")
# Query the same dataset size value using SQL
spark.sql("SELECT count(1) FROM music_reviews").show()

+--------+
|count(1)|
+--------+
|   10261|
+--------+



In [7]:
# Look at two example elements using an SQL query
spark.sql("SELECT * FROM music_reviews limit 2").show()

+----------+--------+-------+--------------------+-----------+--------------+--------------------+-------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+-------+--------------+
|1384719342|  [0, 0]|    5.0|Not much to write...|02 28, 2014|A2IBPI20UZIR0U|cassandra tu "Yea...|   good|    1393545600|
|1384719342|[13, 14]|    5.0|The product does ...|03 16, 2013|A14VAT5EAX3D9S|                Jake|   Jake|    1363392000|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+-------+--------------+



In [28]:
#####################################################
# Challenge! 
#
# Can you use the SparkSQL interface to find the top 
# five reviewers by number of reviews?
#
# Answer:
# +----+--------------+--------------------+-----+
# |rank|    reviewerID|        reviewerName|count|
# +----+--------------+--------------------+-----+
# |   1| ADH0O8UVJOT10|            StormJH1|   42|
# |   2|A15TYOEWBQYF0X|      David W "Dave"|   38|
# |   2|A1L7M2JXN4EZCR|             David G|   38|
# |   4|A2EZWZ8MBEDOLN|              Charlo|   36|
# |   5|A2NYK9KWFMJV4Y|Mike Tarrani "Jaz...|   34|
# +----+--------------+--------------------+-----+
#####################################################


# Example Solution:
spark.sql("""
          select rank, reviewerID, reviewerName, count 
            from
         (select *, rank() over (order by count desc) as rank
            from
         (select reviewerID, reviewerName, count(1) as count
            from music_reviews 
        group by reviewerID, reviewerName) foo) bar
           where rank between 1 and 5
          """).show()

+----+--------------+--------------------+-----+
|rank|    reviewerID|        reviewerName|count|
+----+--------------+--------------------+-----+
|   1| ADH0O8UVJOT10|            StormJH1|   42|
|   2|A15TYOEWBQYF0X|      David W "Dave"|   38|
|   2|A1L7M2JXN4EZCR|             David G|   38|
|   4|A2EZWZ8MBEDOLN|              Charlo|   36|
|   5|A2NYK9KWFMJV4Y|Mike Tarrani "Jaz...|   34|
+----+--------------+--------------------+-----+



In [29]:
#####################################################
# Challenge! 
#
# Can you use the SparkSQL interface to find the 
# distribution of reviews across the five star categories
# (1 through 5) for the top reviewer, StormJH1?
#
# Answer:
# +------------+-----+-----+
# |reviewerName|stars|count|
# +------------+-----+-----+
# |    StormJH1|  5.0|   33|
# |    StormJH1|  4.0|    7|
# |    StormJH1|  3.0|    2|
# +------------+-----+-----+
#####################################################

# Example Solution:
spark.sql("""select reviewerName, overall as stars, count(1) as count 
               from music_reviews
              where reviewerName='StormJH1'
           group by reviewerName, overall
           order by overall desc""").show()

+------------+-----+-----+
|reviewerName|stars|count|
+------------+-----+-----+
|    StormJH1|  5.0|   33|
|    StormJH1|  4.0|    7|
|    StormJH1|  3.0|    2|
+------------+-----+-----+

